<a href="https://colab.research.google.com/github/cedro3/stylegan2/blob/master/Edit_new_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Edit_new_image

---



# 1.Set up
指定環境である **tensorflow 1.14** に変更し、それに伴い **keras 2.0.8** に変更します。そして、githubからコードをダウンロードします。


In [ ]:
!pip install tensorflow==1.14
!pip install tensorflow-gpu==1.14
!pip install keras==2.0.8

In [ ]:
!git clone https://github.com/cedro3/stylegan2.git

In [ ]:
cd stylegan2

#2.Define Function
コードを実行すると、必要なクラスと関数を定義します。\
＊新規画像を作成するクラスと関数も一緒に定義しています。

In [ ]:
import numpy as np
import scipy.ndimage
import os
import PIL.Image
import sys
import bz2
from keras.utils import get_file
import dlib
import argparse
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import re
import projector
import pretrained_networks
from training import dataset
from training import misc
import matplotlib.pyplot as plt
from tqdm import trange

class LandmarksDetector:
    def __init__(self, predictor_model_path):
        """
        :param predictor_model_path: path to shape_predictor_68_face_landmarks.dat file
        """
        self.detector = dlib.get_frontal_face_detector() # cnn_face_detection_model_v1 also can be used
        self.shape_predictor = dlib.shape_predictor(predictor_model_path)

    def get_landmarks(self, image):
        img = dlib.load_rgb_image(image)
        dets = self.detector(img, 1)

        for detection in dets:
            face_landmarks = [(item.x, item.y) for item in self.shape_predictor(img, detection).parts()]
            yield face_landmarks

def image_align(src_file, dst_file, face_landmarks, output_size=1024, transform_size=4096, enable_padding=True):
        # Align function from FFHQ dataset pre-processing step
        # https://github.com/NVlabs/ffhq-dataset/blob/master/download_ffhq.py
        src_file
        dst_file 
        face_landmarks
        output_size=1024 
        transform_size=4096 
        enable_padding=True
            
        lm = np.array(face_landmarks)
        lm_chin          = lm[0  : 17]  # left-right
        lm_eyebrow_left  = lm[17 : 22]  # left-right
        lm_eyebrow_right = lm[22 : 27]  # left-right
        lm_nose          = lm[27 : 31]  # top-down
        lm_nostrils      = lm[31 : 36]  # top-down
        lm_eye_left      = lm[36 : 42]  # left-clockwise
        lm_eye_right     = lm[42 : 48]  # left-clockwise
        lm_mouth_outer   = lm[48 : 60]  # left-clockwise
        lm_mouth_inner   = lm[60 : 68]  # left-clockwise

        # Calculate auxiliary vectors.
        eye_left     = np.mean(lm_eye_left, axis=0)
        eye_right    = np.mean(lm_eye_right, axis=0)
        eye_avg      = (eye_left + eye_right) * 0.5
        eye_to_eye   = eye_right - eye_left
        mouth_left   = lm_mouth_outer[0]
        mouth_right  = lm_mouth_outer[6]
        mouth_avg    = (mouth_left + mouth_right) * 0.5
        eye_to_mouth = mouth_avg - eye_avg

        # Choose oriented crop rectangle.
        x = eye_to_eye - np.flipud(eye_to_mouth) * [-1, 1]
        x /= np.hypot(*x)
        x *= max(np.hypot(*eye_to_eye) * 2.0, np.hypot(*eye_to_mouth) * 1.8)
        y = np.flipud(x) * [-1, 1]
        c = eye_avg + eye_to_mouth * 0.1
        quad = np.stack([c - x - y, c - x + y, c + x + y, c + x - y])
        qsize = np.hypot(*x) * 2

        # Load in-the-wild image.
        if not os.path.isfile(src_file):
            print('\nCannot find source image. Please run "--wilds" before "--align".')
            return
        img = PIL.Image.open(src_file)

        # Shrink.
        shrink = int(np.floor(qsize / output_size * 0.5))
        if shrink > 1:
            rsize = (int(np.rint(float(img.size[0]) / shrink)), int(np.rint(float(img.size[1]) / shrink)))
            img = img.resize(rsize, PIL.Image.ANTIALIAS)
            quad /= shrink
            qsize /= shrink

        # Crop.
        border = max(int(np.rint(qsize * 0.1)), 3)
        crop = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        crop = (max(crop[0] - border, 0), max(crop[1] - border, 0), min(crop[2] + border, img.size[0]), min(crop[3] + border, img.size[1]))
        if crop[2] - crop[0] < img.size[0] or crop[3] - crop[1] < img.size[1]:
            img = img.crop(crop)
            quad -= crop[0:2]

        # Pad.
        pad = (int(np.floor(min(quad[:,0]))), int(np.floor(min(quad[:,1]))), int(np.ceil(max(quad[:,0]))), int(np.ceil(max(quad[:,1]))))
        pad = (max(-pad[0] + border, 0), max(-pad[1] + border, 0), max(pad[2] - img.size[0] + border, 0), max(pad[3] - img.size[1] + border, 0))
        if enable_padding and max(pad) > border - 4:
            pad = np.maximum(pad, int(np.rint(qsize * 0.3)))
            img = np.pad(np.float32(img), ((pad[1], pad[3]), (pad[0], pad[2]), (0, 0)), 'reflect')
            h, w, _ = img.shape
            y, x, _ = np.ogrid[:h, :w, :1]
            mask = np.maximum(1.0 - np.minimum(np.float32(x) / pad[0], np.float32(w-1-x) / pad[2]), 1.0 - np.minimum(np.float32(y) / pad[1], np.float32(h-1-y) / pad[3]))
            blur = qsize * 0.02
            img += (scipy.ndimage.gaussian_filter(img, [blur, blur, 0]) - img) * np.clip(mask * 3.0 + 1.0, 0.0, 1.0)
            img += (np.median(img, axis=(0,1)) - img) * np.clip(mask, 0.0, 1.0)
            img = PIL.Image.fromarray(np.uint8(np.clip(np.rint(img), 0, 255)), 'RGB')
            quad += pad[:2]

        # Transform.
        img = img.transform((transform_size, transform_size), PIL.Image.QUAD, (quad + 0.5).flatten(), PIL.Image.BILINEAR)
        if output_size < transform_size:
            img = img.resize((output_size, output_size), PIL.Image.ANTIALIAS)

        # Save aligned image.
        img.save(dst_file, 'PNG')


def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path


def project_real_images(): 
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    dataset_name = 'dataset'  
    data_dir = 'my'  
    num_images =  6 
    num_snapshots = 5

    print('Loading networks from "%s"...' % network_pkl)
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    proj = projector.Projector()
    proj.set_network(Gs)

    print('Loading images from "%s"...' % dataset_name)
    dataset_obj = dataset.load_dataset(data_dir=data_dir, tfrecord_dir=dataset_name, max_label_size=0, repeat=False, shuffle_mb=0)
    assert dataset_obj.shape == Gs.output_shape[1:]

    os.makedirs('my/real_images', exist_ok=True)  
    for image_idx in range(num_images):
        print('Projecting image %d/%d ...' % (image_idx, num_images))
        images, _labels = dataset_obj.get_minibatch_np(1)
        images = misc.adjust_dynamic_range(images, [0, 255], [-1, 1])
        
        targets=images
        png_prefix='./my/real_images/image'+str(image_idx)
        num_snapshots=num_snapshots
                
        snapshot_steps = set(proj.num_steps - np.linspace(0, proj.num_steps, num_snapshots, endpoint=False, dtype=int))
        misc.save_image_grid(targets, png_prefix + 'target.png', drange=[-1,1])
        proj.start(targets)
        while proj.get_cur_step() < proj.num_steps:
            print('\r%d / %d ... ' % (proj.get_cur_step(), proj.num_steps), end='', flush=True)
            proj.step()
            if proj.get_cur_step() in snapshot_steps:
                misc.save_image_grid(proj.get_images(), png_prefix + 'step%04d.png' % proj.get_cur_step(), drange=[-1,1])
            
            if proj.get_cur_step() == proj.num_steps:  
                vec = proj.get_dlatents() 
                if image_idx == 0:
                   vec_syn = vec
                else:
                   vec_syn = np.concatenate([vec_syn, vec])  
                print(vec_syn.shape)  
        print('\r%-30s\r' % '', end='', flush=True)

    return vec_syn

def generate_real_gif(vec_syn):
    vec0 = vec_syn[0].reshape(1, 18, 512)
    vec1 = vec_syn[1].reshape(1, 18, 512)
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True 
    Gs_syn_kwargs.truncation_psi = 0.5

    image_gif = []
    image_gif_512 = []
    image_gif_256 = []
    os.makedirs('my/real_gif', exist_ok=True)    
    for i in range(40):
        vec = vec0+(vec1-vec0)*i/39
        images =  Gs.components.synthesis.run(vec, **Gs_syn_kwargs) 
        image_one = PIL.Image.fromarray(images[0], 'RGB')
        image_gif.append(image_one)
        image_gif_512.append(image_one.resize((512,512))) 
        image_gif_256.append(image_one.resize((256,256))) 

    image_gif[0].save('./my/real_gif/movie.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0)
    image_gif_512[0].save('./my/real_gif/movie_512.gif', save_all=True, append_images=image_gif_512[1:],
                      duration=100, loop=0)    
    image_gif_256[0].save('./my/real_gif/movie_256.gif', save_all=True, append_images=image_gif_256[1:],
                      duration=100, loop=0)
    print('complete !')


def generate_images_from_style(all_w):
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'  
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
    noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = True

    fig = plt.figure(figsize=(30, 40))
    os.makedirs('my/generate_pic', exist_ok=True)    
    for i in trange(len(all_w)):
        images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs)
        PIL.Image.fromarray(images[i], 'RGB').save('./my/generate_pic/'+str(i)+'.png')     
 
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[i])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()


def style_mixing(vec_syn, col_styles, truncation_psi):  
    network_pkl = 'gdrive:networks/stylegan2-ffhq-config-f.pkl'
    row_seeds = [1, 2, 3]
    col_seeds = [4, 5, 6]
    col_styles = col_styles
    truncation_psi = truncation_psi
    minibatch_size = 4

    # Loading networks 
    _G, _D, Gs = pretrained_networks.load_networks(network_pkl)

    Gs_syn_kwargs = dnnlib.EasyDict()
    Gs_syn_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_syn_kwargs.randomize_noise = False
    Gs_syn_kwargs.minibatch_size = minibatch_size

    # Generating W vectors
    all_seeds = list(row_seeds + col_seeds)   
    all_w = vec_syn[:6]
    w_dict = {seed: w for seed, w in zip(all_seeds, list(all_w))} # [layer, component]

    print('Generating images...') 
    all_images = Gs.components.synthesis.run(all_w, **Gs_syn_kwargs) # [minibatch, height, width, channel]
    image_dict = {(seed, seed): image for seed, image in zip(all_seeds, list(all_images))}

    print('Generating style-mixed images...')
    for row_seed in row_seeds:
        for col_seed in col_seeds:
            w = w_dict[row_seed].copy()
            w[col_styles] = w[col_styles] + (w_dict[col_seed][col_styles]-w[col_styles])*truncation_psi
            image = Gs.components.synthesis.run(w[np.newaxis], **Gs_syn_kwargs)[0]
            image_dict[(row_seed, col_seed)] = image

    print('Saving images...')
    os.makedirs('my/stylemix_images', exist_ok=True)
    for (row_seed, col_seed), image in image_dict.items():
        PIL.Image.fromarray(image, 'RGB').save('./my/stylemix_images/'+str(row_seed)+'-'+str(col_seed)+'.png')

    print('Saving image grid...')
    _N, _C, H, W = Gs.output_shape
    canvas = PIL.Image.new('RGB', (W * (len(col_seeds) + 1), H * (len(row_seeds) + 1)), 'black')

    r, c = 4, 4  # スクリーン設定（4行×4列）
    fig, axs = plt.subplots(r, c, figsize=(16,16), subplot_kw=({'xticks':(),'yticks':()}))

    for row_idx, row_seed in enumerate([None] + row_seeds):
        for col_idx, col_seed in enumerate([None] + col_seeds):
            if row_seed is None and col_seed is None:
                continue
            key = (row_seed, col_seed)
            if row_seed is None:
                key = (col_seed, col_seed)
            if col_seed is None:
                key = (row_seed, row_seed)
            canvas.paste(PIL.Image.fromarray(image_dict[key], 'RGB'), (W * col_idx, H * row_idx)) 

            # スクリーンに画像配置            
            image_plt = np.array(image_dict[key])
            axs[row_idx, col_idx].imshow(image_plt)
            if row_seed is None:
                x, y = col_seed, col_seed
            elif col_seed is None:
                x, y = row_seed, row_seed
            else:
                x, y = row_seed, col_seed
            axs[row_idx, col_idx].set_xlabel(str(x)+'-'+str(y))

    canvas.save('./my/stylemix_images/grid.png') 

    # スクリーン表示
    black = np.zeros((1024,1024,3))  # 黒画像作成
    axs[0,0].imshow(black)
    axs[0,0].axis('off')
    plt.show()
    plt.close()   


# make dirctory
os.makedirs('my/img', exist_ok=True)
os.makedirs('my/al_img', exist_ok=True)
os.makedirs('my/pic', exist_ok=True)
os.makedirs('my/vectors', exist_ok=True)

#3.load_vectors
　Sampleデータをロードします。

In [ ]:
vec_direction = np.load('Sample/vectors/vec_direction.npy')
vec_smile = np.load('Sample/vectors/vec_smile.npy')
vec_glass = np.load('Sample/vectors/vec_glass.npy')
vec_young = np.load('Sample/vectors/vec_young.npy')
vec_old = np.load('Sample/vectors/vec_old.npy')
vec_man = np.load('Sample/vectors/vec_man.npy')

#4.show_generate_images
指定したベクトル(潜在変数w)から生成した画像を表示します。(初回実行時は時間が掛かりますが、２回目以降は早いです）\
＊生成した画像は、my/generate_picに上書き保存します。

In [ ]:
generate_images_from_style(vec_direction)

#5.Style_Mixing
指定された潜在変数wをRow_picとCol_picで入れ替えます。\
第１引数：ベクトル名\
第２引数：潜在変数wの指定（リスト形式）\
第３引数：Col_picの潜在変数の寄与度(0〜1はRow_colとの混合割合)\
＊第２引数、第３引数を色々変更して変化を見ると面白いです。\
＊生成した画像は、my/stylemix_images に上書き保存します



In [ ]:
# Face_direction
style_mixing(vec_direction, [0,1], 1.4)

In [ ]:
# Smile
style_mixing(vec_smile, [4,5], 1.0)

In [ ]:
# Glass
style_mixing(vec_glass, [0,1,2], 1.0)

In [ ]:
# Young
style_mixing(vec_young, [4,5,6,7], 0.8)

In [ ]:
# Old
style_mixing(vec_old, [4,5,6,7], 0.6)

In [ ]:
# Man
style_mixing(vec_man, [4,5,6,7], 1.0)

#Appendix
# ------ 以下は自分で新規画像のベクトルデータを作成するためのもの ------

#10.Get target image from original image
オリジナル画像を **my/imgフォルダーへアップロードしてから**、下記コードを実行する。\
切り出された顔画像は **my/al_imgフォルダー**へ保存される。

In [ ]:
LANDMARKS_MODEL_URL = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
landmarks_model_path = unpack_bz2(get_file('shape_predictor_68_face_landmarks.dat.bz2',
                                            LANDMARKS_MODEL_URL, cache_subdir='temp'))
RAW_IMAGES_DIR = 'my/img'
ALIGNED_IMAGES_DIR = 'my/al_img'

landmarks_detector = LandmarksDetector(landmarks_model_path)
for img_name in os.listdir(RAW_IMAGES_DIR):
    raw_img_path = os.path.join(RAW_IMAGES_DIR, img_name)
    for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(raw_img_path), start=1):
        face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
        aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)
        image_align(raw_img_path, aligned_face_path, face_landmarks)

#11.Make TFRecord dataset from target image
潜在変数wを探索する画像を **my/al_imgフォルダー**から**６つ選択して my/picフォルダーへ\
ドラッグ＆ドロップしてから**、下記コードを実行する。\
実行後マルチ解像度のデータセットは **my/datasetフォルダー**に保存される。

In [ ]:
!python dataset_tool.py create_from_images ./my/dataset ./my/pic

#12.Find the matching latent vector 
コードを実行するとmy/datasetフォルダーにあるマルチ解像度のデータセットから潜在変数wを探索する。\
途中経過の画像は **my/real_imagesフォルダー**に保存される。\
探索した６つの潜在変数は**vec_syn**で返って来る。

In [ ]:
vec_syn = project_real_images()

#13.Make animation from latent vector
コードを実行すると、**変数vec_syn**に格納されている先頭の２つの潜在変数からアニメーションを作成します。\
作成した gifアニメーションは**my/real_gifフォルダー**に保存されます。

In [ ]:
generate_real_gif(vec_syn)

#14.Save vectors
作成したベンクトルデータを保存します。\
＊保存先は、my/vectors です。

In [ ]:
np.save('my/vectors/vec_syn', vec_syn)